In [ ]:
'''
Log:
* 2017/08/21: 3 layer NN not working well with raw wave input (random offset + random gain), trying again with everyhing
  the same except only with samples with random gain
* 2017/08/22: Fix a bug and currently 3 layer NN with 500 hidden layer has testing accuracy of 96.8%:
  When separating data into train, validate and test set need to shuffle first so that each piece of data
  have roughly samples with every classes in it.
* 2017/08/22: Tried to catgorize combinations of 7 sounds but failed to generalize (correct rate is 0.011 which is odd)
* 2017/08/23: Found the bug for the low accuracy: samples are not shuffled. Tried categorizing mixed sample but failed again
* 2017/08/24: If limit mixing offset to 5 frames the accurracy up'd to 80%. Can try to use frequency domain samples or more complex model
* 2017/08/25: Try using spectrogram but the data size is too big. Need to reduce it down a bit
* 2017/08/27: Use mel spectrum as features: offset and amplitude use linear and neural network both achieves >90% accurary and less iteration
* 2017/08/30: Use more samples (65000) and mel spectrum and fix a weight initialization bug NN can achieve 98% test accuracy
* 2017/08/31: Need to do code cleanup to make collecting and organizing samples easier
* 2017/09/04: system test samples don't have good results; it seems that something wrong in the conversion
* 2017/09/06: conversion seems find since the spectrogram shows simular pattern. But It seems that the mixed samples are not recognizable (0 or 63)
'''

In [ ]:
import os
import glob
import librosa
import random
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import time
from pydub import AudioSegment
from scipy.special import comb
import scipy.io.wavfile
import math
import multiprocessing
from random import shuffle
from threading import Thread
%matplotlib inline


# directories
work_dir = os.getcwd()import os
import glob
import librosa
import random
import numpy as np
import librosa.display
import matplotlib.pyplot as plt
import time
from pydub import AudioSegment
source_dir = os.path.join(work_dir, "audio_data")
sample_dir = os.path.join(work_dir, "samples")
fdomain_dir = os.path.join(work_dir, "fdomain")
systemtest_source_dir = os.path.join(work_dir, "systemtest-audio-data")
systemtest_samples = os.path.join(work_dir, "systemtest-samples")
systemtest_fdomain_samples = os.path.join(work_dir, "systemtest-fdomain-samples")

if not os.path.exists(sample_dir):
    os.mkdir(sample_dir)

if not os.path.exists(fdomain_dir):
    os.mkdir(fdomain_dir)


In [ ]:
#
# Utilities to organize and generate samples
#

def ClipAudio(input_dir, dest_dir, length=3000):
    for f in glob.glob(os.path.join(dest_dir, "*.wav")):
        os.remove(f)

    for f in glob.glob(os.path.join(input_dir, "*.wav")):
        sound = AudioSegment.from_file(f, format="wav")
        clipped = sound[0:length]
        out_file_name = os.path.join(dest_dir, os.path.splitext(os.path.basename(f))[0])
        clipped.export(out_file_name + ".wav")
        print("src:", f, "dst:", out_file_name)
    
def PrepareDataSets(directory, train_div, val_div, is_time_domain):   
    start = time.time()
    # load 7 class samples, convert to one-hot encoding    
    full_data, full_label = PrepareData(directory, is_time_domain);
    print("full_data", full_data.shape, "full_label", full_label.shape)

    num_total = full_data.shape[0]
    num_dim = full_data.shape[1]
    assert num_total == full_label.shape[0]

    #separate 65% training set, 15% validation set, %20 testing set
    num_train = int(num_total * train_div)
    num_val = int(num_total * val_div)
    num_test = int(num_total - num_train - num_val)

    training_set = full_data[:num_train]
    validation_set = full_data[num_train:num_train + num_val]
    test_set = full_data[num_train + num_val:]    
    print("training_set", training_set.shape, "validation_set", validation_set.shape, "test_set", test_set.shape)
    
    training_label = full_label[:num_train]
    validation_label = full_label[num_train:num_train + num_val]
    test_label = full_label[num_train + num_val:]    
    print("training_label", training_label.shape, "validation_label", validation_label.shape, "testing_label", test_label.shape)    
    print("PrepareDataSetsTDomain. time: ", time.time() - start)
    return training_set, training_label, validation_set, validation_label, test_set, test_label


def OneHotEncoding(dim, val):
    count = val.shape[0]
    one_hot = np.zeros((count, dim))
    one_hot[np.arange(count), val] = 1
    return one_hot

def LoadAudioFile(wave_names, sounds):            
    files = glob.glob(os.path.join(source_dir, "*.wav"))
    files.sort()
    max_length = 0
    for f in files:
        sound = AudioSegment.from_file(f, format="wav")
        sounds.append(sound)
        name = os.path.basename(f)
        wave_names.append(name[:name.index('.')])
        #print("wave: %s loaded %d samples" % (name, len(sound)))
        max_length = max(max_length, len(sound))
    return max_length

# generate wave sample
def GenerateRandomMixedSamples(wave_names_in, sounds_in, num, length = 3000):    
    files = glob.glob(os.path.join(sample_dir, "*.wav"))
    for f in files:
        os.remove(f)

    assert len(wave_names_in) == len(sounds_in)
    src_wav_cnt = len(wave_names_in)
            
    #exlode mixing comnbinatin
    total_combi = 2 ** src_wav_cnt
    for i in range(1, total_combi):
        print("GenerateRandomMixedSamples(). combi:", i);
        for j in range(num):
            mix = AudioSegment.silent(duration=length * 2)
            max_start = 0
            for k in range(src_wav_cnt):
                if i & (1 << k) != 0:
                    start = random.randrange(0, length / 2)                    
                    #print("start", start)
                    mix = mix.overlay(sounds_in[k], position=start, loop=True)
            #get from the end
            mix = mix[-length:]
            gain = random.randrange(-10.0, 10.0)
            mix = mix.apply_gain(gain)
            mix.export(os.path.join(sample_dir, str(i)) + "-" + str(j) + ".wav", format="wav")
            mix.close()                
    return

def ConvertTDomainToFDomain(src_files, dest_dir):
    for file_name in src_files:
        #print("ConvertTDomainToFDomain", file_name)
        [z,sr]  = librosa.core.load(file_name, sr=8000) 
        # @todo hack to keep array size the same
        y = z[0:24000]
        #print("shape:", y.shape, "sr:", sr)
        # @todo remove hack
        # @todo: tune parameters
        specto = librosa.feature.melspectrogram(y, sr=sr, n_fft=320, hop_length=160, n_mels=80) 
        #log_specto = librosa.core.logamplitude(specto) 
        #plt.figure(figsize=(12,4)) 
        #librosa.display.specshow(log_specto,sr=sr,x_axis='time', y_axis='mel', hop_length=160) 
        #plt.title('mel power spectrogram') 
        #plt.colorbar(format='%+02.0f dB') 
        #plt.tight_layout() 
        #plt.show() 
        #print specto.shape 
        #print log_specto.shape
        fdomain_file_name = os.path.join(dest_dir, os.path.splitext(os.path.basename(file_name))[0])
        #print("name:", fdomain_file_name, "spectro shape", specto.shape)
        np.save(fdomain_file_name, specto.flatten())

def PlotSpectrum(wave_file):
    y, sr = librosa.core.load(wave_file, sr=8000)
    y = y[0:24000]
    specto = librosa.feature.melspectrogram(y, sr=sr, n_fft=320, hop_length=160, n_mels=80) 
    log_specto = librosa.core.logamplitude(specto) 
    plt.figure(figsize=(12,4)) 
    plt.title(os.path.basename(wave_file))
    librosa.display.specshow(log_specto,sr=sr,x_axis='time', y_axis='mel', hop_length=160) 
    plt.show()         

# from time domain samples to generate frequency domain samples
def ConvertSampleToFDomain(src_dir, dest_dir):
    # clean current sample
    #print("ConvertSampleToFDomain src:", src_dir, "dest:", dest_dir)
    files = glob.glob(os.path.join(dest_dir, "*.npy"))
    for f in files:
        os.remove(f)

    start = time.time()
    files = glob.glob(os.path.join(src_dir, "*.wav"))
    file_cnt = len(files)
    ConvertTDomainToFDomain(files, dest_dir)
    print("ConvertSampleToFDomain time elapse: %d", time.time() - start)

def LoadSamples(directory, is_time_domain):
    start = time.time()
    if is_time_domain:
        files = glob.glob(os.path.join(directory, "*.wav"))
    else:
        files = glob.glob(os.path.join(directory, "*.npy"))
    shuffle(files)
    file_cnt = len(files)
    X = []
    y = []
    labels = []
    for i in range(file_cnt):
        file_name = files[i]
        if is_time_domain:
            _, samples = scipy.io.wavfile.read(file_name)
        else:
            samples = np.load(file_name)
        #print(samples.shape)
        #print(os.path.basename(file_name), samples.shape)
        token = os.path.basename(file_name).split('_')[0]
        #print("token:", token, labels)
        correct_class = -1
        if token in labels:
            correct_class = labels.index(token)
            #print("correct class:", correct_class)
        else:
            correct_class = len(labels)
            #print("new class: ", correct_class, " label: ", token)
            labels.append(token)            
        y.append(correct_class)
        X.append(samples)
        
    for i in range(len(labels)):
        print("new class: ", i, " label: ", labels[i])
    print("LoadSamples, time:", time.time() - start)
    return X, y, len(labels)

    
# plot frequency domain samples
def PlotFDomainSamples(directory, number_to_plot):
    # load and plot and check if they match
    files = glob.glob(os.path.join(directory, "*.npy"))
    plt.figure(figsize=(40,60))
    files.sort()
    file_cnt = min(len(files), number_to_plot)
    for i in range(file_cnt):
        file_name = files[i]
        D_loaded = np.load(file_name)
        log_power_loaded = librosa.logamplitude(D_loaded**2, ref_power=np.max)
        plt.subplot(file_cnt, 1, i + 1)
        librosa.display.specshow(log_power_loaded, x_axis="time", y_axis='log')
    plt.show()
    
# Generate num_samples with random amplitude
def GenerateSampleRandomAmp(wave_names_in, sounds_in, num_samples, length):
    files = glob.glob(os.path.join(sample_dir, "*.wav"))
    for f in files:
        os.remove(f)

    assert len(wave_names_in) == len(sounds_in)
    samples_cnt_in = len(wave_names_in)
    for i in range(samples_cnt_in):
        s = sounds_in[i]
        audio_len = len(s)
        repeat_audio = s
        while (len(repeat_audio) < 2 * length):
            repeat_audio = repeat_audio + s
            
        for j in range(num_samples):
            start = 0
            end = start + length
            new_name = os.path.join(sample_dir, wave_names_in[i] + "-" + str(j) + ".wav")
            #print("new_name: %s(%d), start: %d, end: %d" % (os.path.basename(new_name), len(repeat_audio), start, end))
            sj = repeat_audio[start:end]
            gain = random.randrange(-10.0, 10.0)
            sj = sj.apply_gain(gain)
            sj.export(new_name, format="wav")

    
# generate num_samples with random start time and amplitude
def GenerateSampleRandomAmpAndOffset(wave_names_in, sounds_in, num_samples, length):
    files = glob.glob(os.path.join(sample_dir, "*.wav"))
    for f in files:
        os.remove(f)

    assert len(wave_names_in) == len(sounds_in)
    samples_cnt_in = len(wave_names_in)
    for i in range(samples_cnt_in):
        s = sounds_in[i]
        audio_len = len(s)
        repeat_audio = s
        while (len(repeat_audio) < 2 * length):
            repeat_audio = repeat_audio + s
            
        for j in range(num_samples):
            start = random.randrange(0, audio_len)
            end = start + length
            new_name = os.path.join(sample_dir, wave_names_in[i] + "-" + str(j) + ".wav")
            #print("new_name: %s(%d), start: %d, end: %d" % (os.path.basename(new_name), len(repeat_audio), start, end))
            sj = repeat_audio[start:end]
            gain = random.randrange(-10.0, 10.0)
            sj = sj.apply_gain(gain)
            sj.export(new_name, format="wav")

def PrepareData(directory, is_time_domain):
    start = time.time()
    sample, label, num_classes = LoadSamples(directory, is_time_domain);
    print("num_classes", num_classes)
    assert len(sample) == len(label)
    x = np.array(sample)
    y = np.array(label)
    y = OneHotEncoding(num_classes, y)
    print("PrepareData(), time: ", time.time() - start)
    return x, y  


In [ ]:
#
# Generate time domain samples
#
random.seed()


# random samples
#wave_names = []
#sounds = []
#max_length = 0
#max_length = LoadAudioFile(wave_names, sounds)
#print("max_length=%d" % (max_length))
#ConvertSampleToFDomain(sample_dir, fdomain_dir)
systest_set, systest_label = PrepareData(fdomain_dir, False)
print("systest_set",systest_set.shape, "systest_label", systest_label.shape)

#PlotSpectrum("/home/kxhuan/Documents/code/github/sound_classification/samples-random-mix/2-0.wav")
#files = glob.glob("/home/kxhuan/Documents/code/github/sound_classification/systemtest-audio-data/*.wav")
files = glob.glob(os.path.join(sample_dir, "*.wav"))
for i in range (10):
    f = files[i]
    PlotSpectrum(f)

# current used samples
#GenerateRandomMixedSamples(wave_names, sounds, 500, 3000)
#ConvertSampleToFDomain(sample_dir, fdomain_dir)

# random samples with mixes
#GenerateSampleRandomAmpAndOffset(wave_names, sounds, 10000, 3000)
#GenerateSampleRandomAmp(wave_names, sounds, 1, 3000)
#GenerateRandomMixedSamples(wave_names, sounds, 2, 3000)
#GenerateRandomMixedSamples(wave_names, sounds, 200, 3000)

#ConvertSampleToFDomain(sample_dir, fdomain_dir)


In [ ]:
#
# Train with time domain samples - NN
#

import time
import tensorflow as tf
start = time.time()

# prepare data set
#training_set, training_label, validation_set, validation_label, testing_set, testing_label = PrepareDataSets(sample_dir, 0.8, 0.1, True)
training_set, training_label, validation_set, validation_label, testing_set, testing_label = PrepareDataSets(fdomain_dir, 0.8, 0.1, False)

# random mix frequency domain
#training_set, training_label, validation_set, validation_label, testing_set, testing_label = PrepareDataSets("/home/kxhuan/Documents/code/github/sound_classification/fdomain-random-mix", 0.8, 0.1, False)
#ystest_set, systest_label = PrepareData(systemtest_fdomain_samples, False)
#print("systest_set",systest_set.shape, "systest_label", systest_label.shape, "training_set", training_set.shape, "training_label", training_label.shape)

# random mix time domain
#training_set, training_label, validation_set, validation_label, testing_set, testing_label = PrepareDataSets("/home/kxhuan/Documents/code/github/sound_classification/samples-random-mix", 0.8, 0.1, True)

print('PrepareDataSetsTDomain time:', time.time() - start)
start = time.time()

dimension = training_set.shape[1]
assert training_set.shape[1] == dimension
assert validation_set.shape[1] == dimension
assert testing_set.shape[1] == dimension
assert systest_set.shape[1] == dimension

num_class = training_label.shape[1]
assert training_label.shape[1] == num_class
assert validation_label.shape[1] == num_class
assert testing_label.shape[1] == num_class
assert systest_label.shape[1] == num_class

num_train = training_set.shape[0]
assert training_set.shape[0] == training_label.shape[0]

num_test = testing_set.shape[0]
assert testing_set.shape[0] == testing_label.shape[0]

num_validation = validation_set.shape[0]
assert validation_set.shape[0] == validation_label.shape[0]

num_systest = systest_set.shape[0]
assert systest_set.shape[0] == systest_label.shape[0]

sd = 1 / np.sqrt(dimension)

hidden_1 = 500
hidden_2 = 500
hidden_3 = 500

with tf.device('/gpu:0'):
    # define model
    x  = tf.placeholder(tf.float32, [None, dimension])
    y_ = tf.placeholder(tf.float32, [None, num_class])

    # NN
    #--------
    # layer 1
    W_1 = tf.Variable(tf.random_normal([dimension, hidden_1], mean = 0, stddev=sd))
    b_1 = tf.Variable(tf.random_normal([hidden_1], mean = 0, stddev=sd))
    y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

    # layer 2
    W_2 = tf.Variable(tf.random_normal([hidden_1, hidden_2], mean = 0, stddev=sd))
    b_2 = tf.Variable(tf.random_normal([hidden_2], mean = 0, stddev=sd))
    y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)

    # layer 3
    W_3 = tf.Variable(tf.random_normal([hidden_2, hidden_3], mean = 0, stddev=sd))
    b_3 = tf.Variable(tf.random_normal([hidden_3], mean = 0, stddev=sd))
    y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)

    #output
    W_output = tf.Variable(tf.random_normal([hidden_3, num_class], mean = 0, stddev=sd))
    b_output = tf.Variable(tf.random_normal([num_class], mean = 0, stddev=sd))
    y_output = tf.matmul(y_3, W_output) + b_output
    
    ''' linear
    W = tf.Variable(tf.random_normal([dimension, num_class], mean = 0, stddev=sd))
    b = tf.Variable(tf.random_normal([num_class], mean = 0, stddev=sd))
    y_output = tf.matmul(x, W) + b
    '''
    # correct label
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_output))
    optimizer = tf.train.AdamOptimizer(5e-4).minimize(cost)
        
    # initialize model
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        # training
        batch_size = 500
        for epoch in range(80):
            epoch_loss = 0
            current = 0
            for _ in range(int(num_train/batch_size)):
                c, _ = sess.run([cost, optimizer], feed_dict={x: training_set[current:current + batch_size], y_: training_label[current:current + batch_size]})
                epoch_loss += c
                current += batch_size
            print('Epoch:', epoch,  'loss:', epoch_loss)                
            if epoch_loss < 1.0:
                break
                
        correct_prediction = tf.equal(tf.argmax(y_output, 1), tf.argmax(y_, 1))
        predict_y = tf.argmax(y_output, 1)
        correct_y = tf.argmax(y_, 1)
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        print("training set accuracy: ", sess.run(accuracy, feed_dict={x: training_set, y_: training_label}))
        print("testing set dump: ", sess.run([predict_y, correct_y], feed_dict={x: training_set[0:10], y_: training_label[0:10]}))
        print("testing set accuracy: ", sess.run(accuracy, feed_dict={x: testing_set, y_: testing_label}))
        print("testing set dump: ", sess.run([predict_y, correct_y], feed_dict={x: testing_set[0:10], y_: testing_label[0:10]}))
        #print("system test set accuracy: ", sess.run(accuracy, feed_dict={x: systest_set, y_: systest_label}))
        #print("system test set dump: ", sess.run([predict_y, correct_y], feed_dict={x: systest_set, y_: systest_label}))
    
    
print('training time:', time.time() - start)

In [ ]:
#
# Train with MNIST
#

import time
import tensorflow as tf

start = time.time()

# prepare data set
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

print('PrepareDataSetsTDomain time:', time.time() - start)
start = time.time()

dimension = 784
num_class = 10
#assert num_train == training_label.shape[0]

hidden_1 = 500
hidden_2 = 500
hidden_3 = 500

with tf.device('/gpu:0'):
    # define model
    x  = tf.placeholder(tf.float32, [None, dimension])
    y_ = tf.placeholder(tf.float32, [None, num_class])

    # NN
    #--------
    # layer 1
    W_1 = tf.Variable(tf.random_normal([dimension, hidden_1]))
    b_1 = tf.Variable(tf.random_normal([hidden_1]))
    y_1 = tf.nn.relu(tf.matmul(x, W_1) + b_1)

    # layer 2
    W_2 = tf.Variable(tf.random_normal([hidden_1, hidden_2]))
    b_2 = tf.Variable(tf.random_normal([hidden_2]))
    y_2 = tf.nn.relu(tf.matmul(y_1, W_2) + b_2)

    # layer 3
    W_3 = tf.Variable(tf.random_normal([hidden_2, hidden_3]))
    b_3 = tf.Variable(tf.random_normal([hidden_3]))
    y_3 = tf.nn.relu(tf.matmul(y_2, W_3) + b_3)

    #output
    W_output = tf.Variable(tf.random_normal([hidden_3, num_class]))
    b_output = tf.Variable(tf.random_normal([num_class]))
    y_output = tf.matmul(y_3, W_output) + b_output

    # correct label
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_output))
    optimizer = tf.train.AdamOptimizer(1e-4).minimize(cost)
    
    # initialize model
    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        # training
        batch_size = 100
        for epoch in range(50):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                training_set, training_label = mnist.train.next_batch(500)
                c, _ = sess.run([cost, optimizer], feed_dict={x: training_set, y_: training_label})
                epoch_loss += c
            print('Epoch loss:', epoch_loss)

        # test
        correct_prediction = tf.equal(tf.argmax(y_output, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        testing_set, testing_label = mnist.test.next_batch(batch_size)
        print("training set accuracy: ", sess.run(accuracy, feed_dict={x: training_set, y_: training_label}))
        print("testing set accuracy: ", sess.run(accuracy, feed_dict={x: testing_set, y_: testing_label}))
    
print('training time:', time.time() - start)

In [ ]:
#
# training with one layer neural network - time domain

import time
import tensorflow as tf

training_set, training_label, validation_set, validation_label, testing_set, testing_label = PrepareDataSetsTDomain(sample_dir, 0.8, 0.1)


train_cnt = training_set.shape[0]
dimension = training_set.shape[1]
num_classes = training_label.shape[1]
assert training_cnt == training_label.shape[0]

batch_size = 100
num_layer1 = 500
num_layer2 = 500
training_rate = 1e-4

start = time.time()

y_ = tf.placeholder(tf.float32, [None, num_classes])

x = tf.placeholder(tf.float32, [None, dimension])
w_0 = tf.Variable(tf.random_normal([dimension, num_classes]))
b_0 = tf.Variable(tf.random_normal([num_classes]))
y_0 = tf.matmul(x, w_0) + b_0

y = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_0)
train = tf.train.AdamOptimizer(training_rate).minimize(y)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


#define label
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    
    #training
    start = 0
    while start < training_cnt:
        end = min(start + batch_size, training_cnt)
        print("training start:", start, "end:", end)
        sess.run(train, feed_dict = {x:training_set[start:end], y_:training_label[start:end]})
        start += batch_size
        
    train_accuracy = sess.run(accuracy, feed_dict = {x:training_set[0:1], y_:training_label[0:1]})
    print(training_accuracy)
        
     
    #train_accuracy = sess.run(y, feed_dict = {x:training_set[0:1], y_:training_label[0:1]})
    #print(training_set[0:1].shape, training_label[0:1], y)
    #validate
    #train_accuracy = sess.run(accuracy, feed_dict = {x:training_set, y_:training_label})
    #print("train accuracy: ", train_accuracy)
    #test_accurracy = sess.run(accuracy, feed_dict= {x:testing_set, y_:testing_label})
    #print("test accuracy: ", test_accurracy)
    #print("total time:", time.time() - start)    

    




'''
with tf.Session() as sess:
    y_ = tf.placeholder(tf.float32, [None, num_classes])
    
    x = tf.placeholder(tf.float32, [None, dimension])
    w_0 = tf.Variable(tf.random_normal([dimension, num_layer1]))
    b_0 = tf.Variable(tf.random_normal([num_layer1]))
    y_0 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
    
    w_1 = tf.Variable(tf.random_normal([num_layer1, num_layer2]))
    b_1 = tf.Variable(tf.random_normal([num_layer2]))
    y_1 = tf.nn.relu(tf.matmul(y_0, w_1) + b_1)
    
    w_2 = tf.Variable(tf.random_normal([num_layer2, num_classes]))
    b_2 = tf.Variable(tf.random_normal([num_classes]))
    y_2 = tf.nn.relu(tf.matmul(y_1, w_2) + b_2)
    
    y = tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_2)
    train = tf.train.AdamOptimizer(training_rate).minimize(y)
    
    #correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    #accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    init = tf.global_variables_initializer()
    sess.run(init)
    
    #training
    start = 0
    while start < training_cnt:
        end = min(start + batch_size, training_cnt)
        print("training start:", start, "end:", end)
        sess.run(train, feed_dict = {x:training_set[start:end], y_:training_label[start:end]})
        start += batch_size
     
    train_accuracy = sess.run(y, feed_dict = {x:training_set[0:1], y_:training_label[0:1]})
    print(training_set[0:1].shape, training_label[0:1], y)
    #validate
    #train_accuracy = sess.run(accuracy, feed_dict = {x:training_set, y_:training_label})
    #print("train accuracy: ", train_accuracy)
    #test_accurracy = sess.run(accuracy, feed_dict= {x:testing_set, y_:testing_label})
    #print("test accuracy: ", test_accurracy)
    #print("total time:", time.time() - start)    
'''




In [ ]:
#
# train with frequency domain under development
#

# generate 7 class samples
random.seed()

#wave names and sound files
wave_names = []
sounds = []
max_length = 0
max_length = LoadAudioFile(wave_names, sounds)
print("max_length=%d" % (max_length))

#generate 2 samples per wave
GenerateSample(wave_names, sounds, 1000, 3000)

#generate frequency domain sample
GenerateFDomainSample(sample_dir, fdomain_dir)

In [ ]:
#Generate real sample under development
random.seed()


#wave names and sound files
wave_names = []
sounds = []
max_length = 0
max_length = LoadAudioFile(wave_names, sounds)
print("max_length=%d" % (max_length))
GenerateRandomMixedSamples(wave_names, sounds, 2)

#generate frequency domain sample
GenerateFDomainSample(sample_dir, fdomain_dir)

#plot some samples
PlotFDomainSamples(fdomain_dir, 10)